In [2]:
import numpy as np
import heapq # min-heap를 구현하는 heap queue
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [9]:
# 지도:  벽~1, 빈 공간~0
grid = np.array([
    [0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,1,0,1],
    [1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0],
    [1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1],
    [0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,0],
    [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,1,1,1,0,0],
    [0,0,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
    [0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
])

start = (0,0)
goal = (0,19)

# 휴리스틱 함수 h() : a,b 사이 유클리드 거리
## 시간이나 정보가 불충분하여 합리적 판단을 할 수 없거나
## 굳이 체계적이고 합리적인 판단을 할 필요가 없는 상황에서
## 신속하게 어림짐작하는 것
def heuristic(a,b):
    return np.sqrt( (b[0]-a[0])**2 + (b[1]-a[1])** 2)

# A* 알고리즘 : 추정한 전체비용을 최소로 하는 노드를 확장해가는 방법
# f(n) : 노드 n을 경유하는 전체 비용, g(n)+h(n)
# g(n) : 노드 n까지 이미 투입된 비용
# h(n) : 목표 노드까지 남은 비용 ~ 정확한 예측이 불가하므로 휴리스틱 함수 h^() 사용
# -> 노드 n을 경유하는 추정 전체 비용 f^(n) = g(n) + h^(n)
def Astar(array, start, goal):
    # 이웃 위치 8방향
    neighbors = [(0,1),(0,-1),(1,0),(-1,0),
                 (1,1),(1,-1),(-1,1),(-1,-1)]
    close_set = set() # 탐색 종료된 위치들의 집합
    came_from = {}
    gscore = {start:0} # 시작 위치의 g()값
    fscore = {start:heuristic(start,goal)} # 시작 위치의 f()값
    oheap = [] # min-heap
    heapq.heappush(oheap, (fscore[start],start)) # (거리,출발지) min-heap에 저장
    
    while oheap:
        current = heapq.heappop(oheap)[1] # f()값이 최소인 노드 추출
        if current == goal: # 목적지 도착 상황
            data = []
            while current in came_from: # 목적지에서 역순으로 경로 추출
                data.append(current)
                current = came_from[current]
            return data
        close_set.add(current) # current 위치를 탐색이 종료된 것으로 간주
        
        for i,j in neighbors: # current 위치의 각 이웃 위치에 대한 f()값 계산
            neighbor = current[0]+i, current[1]+j # 이웃 위치
            if 0 <= neighbor[0] < array.shape[0]:
                if 0 <= neighbor[1] < array.shape[1]:
                    if array[neighbor[0]][neighbor[1]] == 1: # 벽
                        continue
                    else : # y 방향 경계 벗어남
                        continue
                else : # x 방향 경계 벗어남
                    continue
            
            temp_g_score = gscore[current] + heuristic(current, neighbor) # g^(n) = g(c)+h((c,n))
            if neighbor in close_set and temp_g_score >= gscore.get(neighbor,0):
                continue # 이미 방문한 위치이면서 g^()값이 기존 g()값보다 큰 경우 -> 무시
            
            if temp_g_score < gscore.get(neighbor,0) or neighbor not in [i[1] for i in oheap]:
                # g^(n) < g(n) 이거나 n을 처음 방문한 경우
                came_from[neighbor] = current # neighbor에 도달한 최선의 경로에서 직전 위치는 current
                gscore[neighbor] = temp_g_score # g(n) = g^(n)
                fscore[neighbor] = gscore[current] + heuristic(neighbor, goal) # f() = g()+h()
                heapq.heappush(oheap, (fscore[neighbor], neighbor)) # min heap에 (f(), neighbor) 삽입
    return False

In [ ]:
route = Astar(grid, start, goal)
route = route + [start]
route = route[::-1] # 역순으로 변환
print("경로 : ", route)

# route에서 x, y 좌표 추출
x_coords = []
y_coords = []

for i in (range(0,len(route))):
    x = route[i][0]
    y = route[i][1]
    x_coords.append(x)
    y_coords.append(y)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
ax.imshow(grid, cmap=plt.cm.Pastel1)
ax.scatter(start[1],start[0], marker="*", color="red", s=200)
ax.scatter(goal[1],goal[0], marker="*", color="green", s=200)
ax.plot(y_coords, x_coords, color='blue')
plt.show()